#Requirements

In [ ]:
!C:\Users\len\projects\venv\Scripts\python.exe -m pip install --upgrade pip

In [3]:
!pip install -U jupyter

You should consider upgrading via the 'C:\Users\len\projects\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [5]:
!pip install -U ipywidgets

You should consider upgrading via the 'C:\Users\len\projects\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install -r "C:\Users\len\projects\requiements.txt"

In [1]:
# import sys
import os
import gc
import warnings
import random
from copy import deepcopy
import random
import math

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torchvision
from torch import nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch.utils.data import Dataset, DataLoader
from madgrad import MADGRAD, MirrorMADGRAD
from torch.optim import AdamW
from sklearn.metrics import f1_score
from transformers import (
    get_constant_schedule,
    get_constant_schedule_with_warmup,
    get_cosine_schedule_with_warmup,
    get_cosine_with_hard_restarts_schedule_with_warmup,
    get_linear_schedule_with_warmup,
    get_polynomial_decay_schedule_with_warmup
)

from sklearn.model_selection import StratifiedKFold,KFold
    
from tqdm import tqdm

import albumentations as A
warnings.filterwarnings("ignore")
tqdm.pandas()

In [ ]:
PROJECT_DIR = ""

# EDA

In [ ]:
classes = ['Red', 'Green', 'Violet', 'White', 'Yellow', 'Brown', 'Black', 'Blue', 'Cyan', 'Grey', 'Orange']
counts = []
for class_name in classes:
    counts.append(len(os.listdir(f"{PROJECT_DIR}/data/train/{class_name}")))

In [ ]:
help(sns.barplot)

In [ ]:
sns.barplot(classes, counts, palette=classes)
# for i in range(len(classes)):
#     barplot[i].set_color(classes[i])

# Dataset

In [ ]:
classes = ['Red', 'Green', 'Violet', 'White', 'Yellow', 'Brown', 'Black', 'Blue', 'Cyan', 'Grey', 'Orange']
target_encoder = {}
for i in range(len(classes)):
    target_encoder[classes[i]] = i

In [ ]:
import os
import pandas as pd
from torchvision.io import read_image


class CarDataset(Dataset):
    def __init__(self, dir, transform=None, target_encoder: dict = None):
        self.transform = transform
        self.target_encoder = target_encoder
        class_names = os.listdir(dir)
        print(class_names)
        self.filenames = []
        self.labels = []

        for class_name in tqdm(class_names):
            class_path = f"{dir}/{class_name}"
            images = os.listdir(class_path)
            for image in images:
                self.filenames.append(f"{dir}/{class_name}/{image}")
                self.labels.append(class_name)

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = self.filenames[idx]
        image = read_image(img_path)
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)
        
        if self.target_encoder:
            label = self.target_encoder[label]

        return image, label

In [ ]:
train_dataset = CarDataset(f"{PROJECT_DIR}/data/train", target_encoder=target_encoder)

# Losses

In [2]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=0, eps=1e-7):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.eps = eps
        self.ce = torch.nn.CrossEntropyLoss()

    def forward(self, input, target):
        logp = self.ce(input, target)
        p = torch.exp(-logp)
        loss = (1 - p) ** self.gamma * logp
        return loss.mean()


class ArcFaceLoss(nn.Module):
    def __init__(self, s=45.0, m=0.1, crit="bce", weight=None, reduction="mean",
                 focal_loss_gamma=0, class_weights_norm="batch"):
        super().__init__()

        self.weight = weight
        self.reduction = reduction
        self.class_weights_norm = class_weights_norm
        
        if crit == "focal":
            self.crit = FocalLoss(gamma=focal_loss_gamma)
        elif crit == "bce":
            self.crit = nn.CrossEntropyLoss(reduction="none")   

        if s is None:
            self.s = torch.nn.Parameter(torch.tensor([45.], requires_grad=True, device='cuda'))
        else:
            self.s = s

        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m
        
    def forward(self, logits, labels):

        logits = logits.float()
        cosine = logits
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        
        labels2 = torch.zeros_like(cosine)
        labels2.scatter_(1, labels.view(-1, 1).long(), 1)
        labels2 *= (1 - 0.1)
        labels2 += 0.005
        output = (labels2 * phi) + ((1.0 - labels2) * cosine)

        s = self.s

        output = output * s
        loss = self.crit(output, labels)

        if self.weight is not None:
            w = self.weight[labels].to(logits.device)

            loss = loss * w
            if self.class_weights_norm == "batch":
                loss = loss.sum() / w.sum()
            if self.class_weights_norm == "global":
                loss = loss.mean()
            else:
                loss = loss.mean()
            
            return loss
        if self.reduction == "mean":
            loss = loss.mean()
        elif self.reduction == "sum":
            loss = loss.sum()
        return loss

# Modules

In [3]:
class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.weight = nn.Parameter(torch.Tensor(out_features, in_features))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)

    def forward(self, features):
        cosine = F.linear(F.normalize(features), F.normalize(self.weight))
        return cosine

# Model

In [33]:
#model_weights = torch.load('C:/Users/len/projects/model/model.pt', map_location=torch.device('cpu'))
model = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V1)
model.fc = torch.nn.Linear(in_features=2048, out_features=11, bias=True)

# Loops

In [34]:
def train_epoch(model, data_loader, loss_function, optimizer, scheduler, device):
    model.train()
    total_train_loss = 0

    dl_size = len(data_loader)

    batch_i = 0
    preds_copy = torch.tensor([])
    target_copy = torch.tensor([])
    for batch in tqdm(data_loader):
        target_copy = torch.cat((target_copy, batch[1]), 0)
        b_input = batch[0].to(device)
        b_target = batch[1].to(device)

        optimizer.zero_grad()        
        preds = model(b_input)
        preds_copy = torch.cat((preds_copy, preds.to('cpu')), 0)
        loss = loss_function(preds, b_target)
        total_train_loss += loss.item()

        loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
    
    preds_copy = torch.argmax(preds_copy, dim=1)
    epoch_accuracy = f1_score(preds_copy, target_copy, average='macro')

    return ((total_train_loss / dl_size), epoch_accuracy)    
    
    
def eval_epoch(model, data_loader, loss_function, device):
    model.eval()
    total_train_loss = 0

    dl_size = len(data_loader)

    preds_copy = torch.tensor([])
    target_copy = torch.tensor([])
    
    for batch in tqdm(data_loader):
        target_copy = torch.cat((target_copy, batch[1]), 0)
        b_input_ids = batch[0].to(device)
        b_target = batch[1].to(device)

        with torch.no_grad():
            preds = model(b_input_ids)
            preds_copy = torch.cat((preds_copy, preds.to('cpu')),0)

        loss = loss_function(preds, b_target)
        total_train_loss += loss.item()

    preds_copy = torch.argmax(preds_copy, dim=1)
    epoch_accuracy = f1_score(preds_copy, target_copy, average='macro')

    return ((total_train_loss / dl_size), epoch_accuracy)

In [35]:
def save_model(model, path):
    torch.save(model, path+'/model.pt')

In [50]:
def cross_validation(model, 
                     dataset, 
                     loss_function,
                     strat_array, 
                     shuffle: bool=True,
                     device=torch.device("cpu"),
                     random_state: int=69, 
                     n_folds: int=4, 
                     epochs: int=5, 
                     lr: float=1e-4,
                     start_fold: int=0, 
                     batch_size: int=32):
    random.seed(random_state),
    np.random.seed(random_state)
    torch.manual_seed(random_state)
    torch.cuda.manual_seed_all(random_state)
    kfold = StratifiedKFold(n_folds, shuffle=shuffle, random_state=random_state)
    for fold, (train_ids, eval_ids) in enumerate(kfold.split(dataset, strat_array)):
        if fold >= start_fold:
            print(f'FOLD {fold}')
            print('--------------------------------')

            fold_model = deepcopy(model)

            optimizer = torch.optim.AdamW(fold_model.parameters(), lr = 1e-4)
            
            fold_path = f'C:/Users/len/projects/model/saves/fold_{fold}'
            if not os.path.exists(fold_path):
                os.mkdir(fold_path)

            train_subsampler = torch.utils.data.Subset(dataset,  train_ids)
            train_loader = torch.utils.data.DataLoader(
                          train_subsampler, 
                          batch_size=batch_size)

            eval_subsampler = torch.utils.data.Subset(dataset,  eval_ids)
            eval_loader = torch.utils.data.DataLoader(
                          eval_subsampler,
                          batch_size=batch_size)

            total_steps = len(train_loader) * epochs 

            scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                    num_warmup_steps = 0, # Default value in run_glue.py
                                                    num_training_steps = total_steps)

            for epoch_i in range(0, epochs):
                train_loss, train_accuracy = train_epoch(fold_model, train_loader, loss_function, optimizer, scheduler, device)
                eval_loss, eval_accuracy = eval_epoch(fold_model, eval_loader, loss_function, device)

                print(f'Epoch {epoch_i}')
                print(f'Train loss: {train_loss} Train accuracy: {train_accuracy}')
                print(f'Eval loss: {eval_loss} Eval accuracy: {eval_accuracy}')

                epoch_path = fold_path+f'/epoch_{epoch_i}'
                if not os.path.exists(epoch_path):
                    os.mkdir(epoch_path)
                save_model(fold_model,epoch_path)




def single_model(model,
                 dataset, 
                 loss_function, 
                 device=torch.device("cuda"),
                 random_state: int=69, 
                 epochs: int=5, 
                 lr: float=1e-4,
                 batch_size: int=32,
                 start_epoch: int = 0):
    random.seed(random_state),
    np.random.seed(random_state)
    torch.manual_seed(random_state)
    torch.cuda.manual_seed_all(random_state)
    
    loss_function.to(device)
    model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-4)

    data_loader = torch.utils.data.DataLoader(
                    dataset,
                    batch_size=batch_size)
    
    total_steps = len(data_loader) * epochs 

    scheduler = get_cosine_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
    
    fold_path = f'C:/Users/len/projects/model/saves/train'
    if not os.path.exists(fold_path):
        os.mkdir(fold_path)
    losses = []
    accuracies  = []
    for epoch_i in range(0, epochs):
        if epoch_i >= start_epoch:
            epoch_path = f"{fold_path}/epoch_{epoch_i}/"
            os.mkdir(epoch_path)
            
                
            epoch_loss, epoch_accuracy = train_epoch(model, data_loader, loss_function, optimizer, scheduler, device)
            losses.append(epoch_loss)
            accuracies.append(epoch_accuracy)
            print(f'Epoch {epoch_i}')
            print(f'Train loss: {epochs_loss} Train accuracy: {epochs_accuracy}')
            epoch_path = fold_path+f'/epoch_{epoch_i}'
            if not os.path.exists(epoch_path):
                os.mkdir(epoch_path)
            save_model(model,epoch_path)
            
            # eval_epoch(fold_model, eval_loader, loss_function, device)

# Load Data

In [37]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transforms=torchvision.transforms.Compose([
    torchvision.transforms.Resize(size=(256,256)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Lambda(lambda a: a / 255),
    torchvision.transforms.Normalize(mean, std)
])

In [38]:
data_path = 'C:/Users/len/projects/data'
train_folder = torchvision.datasets.ImageFolder(data_path+'/train', transform=transforms)

train_loader = torch.utils.data.DataLoader(train_folder, batch_size=64, shuffle=False, num_workers=1)

In [39]:
strat_array = []
target_class = 0
for folder in os.listdir(data_path+'/train'):
    for _ in range(len(os.listdir(data_path+'/train'+f'/{folder}'))):
        strat_array.append(target_class)
    target_class += 1

In [40]:
criterion = nn.CrossEntropyLoss()
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

# Начальника запускай кросc

In [ ]:
single_model(model = model,
                 dataset = train_folder,
                 loss_function = criterion,
                 device = device)

  3%|████▌                                                                                                                                                 | 3/99 [00:27<14:23,  8.99s/it]

In [42]:
example = torch.rand((64, 11))

In [44]:
torch.argmax(example, dim = 1)

tensor([ 8,  1,  7,  9, 10,  5, 10,  4,  6,  9,  0,  8,  8,  5,  6,  9, 10,  2,
         0,  9,  4,  3,  8,  2,  6,  8, 10,  8,  2,  7,  8,  1, 10,  7, 10,  5,
         0, 10,  9,  1, 10,  7,  6,  5,  9,  3,  0,  7,  1,  1,  0,  9,  2,  3,
         2,  4,  0,  7,  1,  8,  6,  7,  8,  4])

In [20]:
torch.cuda.is_available()

False

In [30]:
!pip install --no-cache-dir --force-reinstall torch===1.7.1

     -------------------------------------- 184.0/184.0 MB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 14.7/14.7 MB 5.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.4
    Uninstalling numpy-1.23.4:
      Successfully uninstalled numpy-1.23.4


ERROR: Could not install packages due to an OSError: [WinError 5] Отказано в доступе: 'C:\\Users\\len\\projects\\venv\\Lib\\site-packages\\~umpy\\.libs\\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll'
Check the permissions.



In [32]:
torch.cuda.is_available()

False

In [1]:
!pip install --upgrade setuptools pip wheel

     ---------------------------------------- 1.2/1.2 MB 3.7 MB/s eta 0:00:00
  Attempting uninstall: wheel
    Found existing installation: wheel 0.36.2
    Uninstalling wheel-0.36.2:
      Successfully uninstalled wheel-0.36.2
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.0.0
    Uninstalling setuptools-57.0.0:
      Successfully uninstalled setuptools-57.0.0


In [2]:
!pip install nvidia-pyindex

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for nvidia-pyindex: filename=nvidia_pyindex-1.0.9-py3-none-any.whl size=8431 sha256=720d44fb15f6c340eddd1a09d4083abb73cd4f9b318f8e74454d6a3b2fc4fc58
  Stored in directory: c:\users\len\appdata\local\pip\cache\wheels\0e\62\68\8bb6aafc3cb47e3468055aebc10d004b55da43563d748aac9c
Successfully built nvidia-pyindex


In [4]:
!pip install nvidia-cuda-runtime-cu11

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------ 991.4/991.4 kB 923.9 kB/s eta 0:00:00
  Created wheel for nvidia-cuda-runtime-cu11: filename=nvidia_cuda_runtime_cu11-2022.4.25-py3-none-any.whl size=15685 sha256=c32f473ffdada748d50fcf0de798dd32a18a1183a9e514da0759755a0aaf609b
  Stored in directory: C:\Users\len\AppData\Local\Temp\pip-ephem-wheel-cache-6_z_bexz\wheels\a4\7d\9f\a63debe4d85b226df3ce0742ac48b5325ad4a5d8af35be35e2
Successfully built nvidia-cuda-runtime-cu11


In [5]:
import torch

In [6]:
torch.cuda.is_available()

False